Normalize ALL DATA using mecan, data are filtered during normalization, and normalized data is stored back to the same record.

In [1]:
import mecan4cna.algorithms as alg
import mecan4cna.common as comm
from pymongo import MongoClient
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
client = MongoClient()

The original testing code

In [11]:
db = client['Rebased']['mecaned']
for sam in db.find():
    if (sam['base'] != None) and (sam['base'] <= 2.3) and (sam['base'] >= 1.7):
        normalized = comm.normalize(sam['segments'], sam['base'], sam['level_distance'])
    else:
        normalized = None
    db.update_one({'sample_id':sam['sample_id']}, {'$set':{'normalized':normalized}}, upsert=True)

A function easy processing

In [3]:
def normalizeDataset(database, collection):
    db = client[database][collection]
    c = 0
    for sam in db.find():
        if 'normalized' not in sam.keys():
            if (sam['base'] != None) and (sam['base'] <= 2.3) and (sam['base'] >= 1.7):
                normalized = comm.normalize(sam['segments'], sam['base'], sam['level_distance'])
            else:
                normalized = None
            try:
                db.update_one({'sample_id':sam['sample_id']}, {'$set':{'normalized':normalized}}, upsert=True)
            except Exception as e:
                print(e)
                print(sam['sample_id'])
        print(c, end='\r')
        c +=1

Normalize all data

In [4]:
normalizeDataset('Rebased', 'mecaned')